In [12]:
import os
import re
import mikeio
import numpy as np
import pandas as pd
import datetime as dt
from res1d_v5 import *
import pickle
import plotly.express as px
import plotly
from plotly.subplots import make_subplots
from IPython.display import clear_output

Brainstorm

Add flood zoomable floodmap
Add 1s result timestep for pump stations


In [13]:
#General variables

period_specs = []
period_specs.append('DWF')
for i in range(1,6):
    period_specs.append('WWF' + str(i))
period_specs

['DWF', 'WWF1', 'WWF2', 'WWF3', 'WWF4', 'WWF5']

In [94]:
#Model specific variables
model_area = "NSSA"

rolling_average = False

#Only for debug then remove
gauges = ['HL14']

summation_csv = r"J:\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports\Summation.csv"
calibration_sheet = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Calibration_Master.xlsm"
rainfall_dfs0_file = r"J:\SEWER_AREA_MODELS\NSSA\03_SIMULATION_WORK\SYSTEM_ASSESSMENT\DATA\RAINFALL\NSSA_Rainfall_Data_PDT.dfs0"

map_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Report_Maps"
output_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports"
result_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\06. CALIB_RESULTS\Latest_Backup49"

dfs0_folders = []
dfs0_folders.append(r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\Non-PS")
dfs0_folders.append(r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\PS")

,Pipe,CSO1,CSO2,Location,Node1 (Or Pipe if pipe level),Node2,CSOPipe1,CSOPipe2,Rain Gauge,Model,Sensor Output,Location Type,Pipe Shape,Pipe Dimension,Node1 AssetName,Flow source,Level Type
Gauge,,,,,,,,,,,,,,,,,
FA2,49582,NaN,NaN,FA2,9600,NaN,NaN,NaN,DN72,NSSA-E,Level and Flow,Gauge,Circular,0.6,MH12A,Measured,Node
GE1,50924,NaN,NaN,GE1,GE1-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,,Measured,Node
GE2,56296,NaN,NaN,GE2,GE2-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,NaN,Measured,Node
GE3,50933,NaN,NaN,GE3,GE3-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
GE4,50940,NaN,NaN,GE5,GE4-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
GE5,50952,NaN,NaN,GE5,GE5-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
GN14,49709,NaN,NaN,GN14,7169,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,Gauge,Circular,0.45,MH14,Measured,Node
HL1,43953,NaN,NaN,HL1,9721,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.9,MH3,Measured,Node
HL14,43945,NaN,NaN,HL14,9712,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.75,MH14,Measured,Node


In [136]:
gauges = pd.read_excel(calibration_sheet,sheet_name="Gauges")
gauges = gauges.loc[:, :'Level Type']
gauges = gauges[gauges.Model.str.contains(model_area)]
gauges.set_index('Gauge',inplace=True)


periods = pd.read_excel(calibration_sheet,sheet_name="Periods")
periods = periods[periods.Model.str.contains(model_area)]

summation_df = pd.read_csv(summation_csv,dtype={'MUID': str,'SUMTO': str})

report_text = pd.read_excel(calibration_sheet,sheet_name="Report_Text")
report_text = report_text[report_text.Model.notna()]
report_text = report_text[report_text.Model.str.contains(model_area)]
report_text


,Zone,DWF Text,WWF Text,Issue Text,Recommendation Text,Model
172,FA2,The flow measurement is spiky. After DWF cali...,FA2 shows good match in total volume and peak ...,Manhole spilling is shown in simulation but ca...,check with local municipality about manhole sp...,NSSA-E
173,LY7,NaN,LY7 combines the flows in LY1 and LY2. The num...,It was found that the previously described con...,The pump setup in the model to match the contr...,NSSA-E
174,MK20,"Due to the lack of reliability with the MK20, ...",DWF calibration is further improved before WWF...,GWI of d/s MK3 meter < GWI of u/s MK20 meter. ...,"Recommend that metering be repeated, at a loca...",NSSA-E
175,MK3,"Due to the lack of reliability with the MK20, ...",MK3 is calibrated together with NV8.\nIn valid...,GWI of d/s MK3 meter < GWI of u/s MK20 meter. ...,Recommend that either the MV MK3 be fixed and ...,NSSA-E
176,NV1,"After DWF calibration, peak level difference a...",NV1 is inconsistent with upstream and downstre...,NaN,NaN,NSSA-E
178,NVLS5,NV19 was used to calibrate for DWF. The per ca...,NVLS1 is located at the outlet chamber. The to...,NVLS5 has no level data.,NaN,NSSA-E
179,NV13,NaN,NV13 has a good match for the October and the ...,NV13 has no data for the two last events.,NaN,NSSA-E
180,NV19,The per capita and per area loading rates have...,NV19 is not calibrated in WWF.,The larger discrepancy between the peak flow r...,NaN,NSSA-E
181,NV20,The per capita and per area loading rates have...,NV20 has a good match in total volumes and pea...,The flow proportionality between NV20 and its ...,"For future metering plans, implementation and ...",NSSA-E
182,NV21,"In terms of field measurements, ADWF of the u/...",NV21 is downstream of New Lynn Siphon and is u...,ADWF of the u/s NV19 is 85.9 L/s which is larg...,Tributary to NV19 and NV21 may benefit from fu...,NSSA-E


In [89]:
gauges.head(50)


,Gauge,Pipe,CSO1,CSO2,Location,Node1 (Or Pipe if pipe level),Node2,CSOPipe1,CSOPipe2,Rain Gauge,Model,Sensor Output,Location Type,Pipe Shape,Pipe Dimension,Node1 AssetName,Flow source,Level Type
223,FA2,49582,NaN,NaN,FA2,9600,NaN,NaN,NaN,DN72,NSSA-E,Level and Flow,Gauge,Circular,0.6,MH12A,Measured,Node
224,GE1,50924,NaN,NaN,GE1,GE1-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,,Measured,Node
225,GE2,56296,NaN,NaN,GE2,GE2-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,NaN,Measured,Node
226,GE3,50933,NaN,NaN,GE3,GE3-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
227,GE4,50940,NaN,NaN,GE5,GE4-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
228,GE5,50952,NaN,NaN,GE5,GE5-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
229,GN14,49709,NaN,NaN,GN14,7169,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,Gauge,Circular,0.45,MH14,Measured,Node
230,HL1,43953,NaN,NaN,HL1,9721,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.9,MH3,Measured,Node
231,HL14,43945,NaN,NaN,HL14,9712,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.75,MH14,Measured,Node
232,HL18,49559,NaN,NaN,HL14,9708,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.55,MH18,Measured,Node


In [17]:
level_pipes = gauges[gauges['Level Type'].str.contains('Link')][['Node1 (Or Pipe if pipe level)','Level Type']].copy()
level_pipes.rename(columns={'Node1 (Or Pipe if pipe level)':'Node'},inplace=True)
level_pipes

,Node,Level Type
241,43987,Link(DS)
253,54852,Link(DS)
254,54850,Link(DS)


In [18]:
nodes1 = gauges[gauges['Level Type']=='Node']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
nodes2 = gauges[gauges['Level Type']=='Node']['Node2'].dropna().unique().flatten().tolist()
level_nodes = nodes1 + nodes2
#Convert all to string
level_nodes = [str(x) for x in level_nodes]
#Make set to clear duplicates
level_nodes = set(level_nodes)

ds_level_pipes = gauges[gauges['Level Type']=='Link(DS)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
us_level_pipes = gauges[gauges['Level Type']=='Link(US)']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
flow_pipes = gauges.Pipe.dropna().unique().flatten().tolist()

#Convert all to string
ds_level_pipes = [str(x) for x in ds_level_pipes]
us_level_pipes = [str(x) for x in us_level_pipes]
flow_pipes = [str(x) for x in flow_pipes]

#Make set to clear duplicates
ds_level_pipes = set(ds_level_pipes)
us_level_pipes = set(us_level_pipes)
flow_pipes = set(flow_pipes)

In [54]:
for ds_level_pipe in ds_level_pipes:
    print(ds_level_pipe)

54852
43987
54850


In [20]:
from mikeio.dfs0 import Dfs0
dfs0_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\Non-PS"

dfs = Dfs0()
res = mikeio.read(dfs0_folder + '\\' 'HL14_20180614_20190403_5min.dfs0')
df = res.to_dataframe()
df.iloc[:,[0,6]]

,HL14_Flow_L,HL14_Level
2018-06-14 00:00:00,105.150002,-0.659200
2018-06-14 00:05:00,106.050003,-0.657905
2018-06-14 00:10:00,108.650002,-0.654010
2018-06-14 00:15:00,110.349998,-0.650690
2018-06-14 00:20:00,110.150002,-0.650770
...,...,...
2019-04-03 23:35:00,109.550003,-0.649750
2019-04-03 23:40:00,110.150002,-0.648520
2019-04-03 23:45:00,109.849998,-0.648040
2019-04-03 23:50:00,108.699997,-0.649165


In [21]:
f = 'HL14_20180614_20190403_5min.dfs0'

ts = mikeio.read(dfs0_folder + '\\' + f).to_dataframe()
gauge_id = re.split('_|.',f)[0]
ts['Gauge'] = gauge_id
ts.rename(columns={ts.columns[1]:'Flow'},inplace=True)
if len(ts.columns)>6:
    ts.rename(columns={ts.columns[6]:'Level'},inplace=True) 
else:
    ts['Level'] = np.nan
ts = ts[['Gauge','Flow','Level']]


In [22]:
#Import rain gauges

rainfall = mikeio.read(rainfall_dfs0_file).to_dataframe()

#Import flow/level gauges
gauge_ids = []
first_dfs0 = True
for dfs0_folder in dfs0_folders:
    for f in os.listdir(dfs0_folder):
        if f[-5:]=='.dfs0':
            gauge_id = re.split(r'_|[.]',f)[0]
            ts = mikeio.read(dfs0_folder + '\\' + f).to_dataframe()
            ts['Gauge'] = gauge_id
            gauge_ids.append(gauge_id)
            ts.rename(columns={ts.columns[0]:'Flow'},inplace=True)
            if len(ts.columns)>5:
                ts.rename(columns={ts.columns[6]:'Level'},inplace=True) 
            else:
                ts['Level'] = np.nan
            ts = ts[['Gauge','Flow','Level']]
            if first_dfs0 == True:
                measured = ts.copy()
            else:
                measured = pd.concat([measured,ts])
            first_dfs0 = False
            
        

In [23]:
# nodes = resultdata.resultData.Nodes
# for i, node in enumerate(nodes):
#     rid = node.Id


#     if rid in level_nodes:
#         print ("Importing node " + node.Id + " at " + str(dt.datetime.now()))
#         for di in node.DataItems:
#             if di.Quantity.Id.lower() == 'waterlevel':



IndentationError: expected an indented block (2035513836.py, line 11)

In [62]:
list(di.CreateTimeSeriesData(2))

[12.31151,
 12.31077,
 12.31001,
 12.30925,
 12.30849,
 12.30771,
 12.30694,
 12.30617,
 12.30546,
 12.30481,
 12.30417,
 12.30355,
 12.30291,
 12.30227,
 12.30163,
 12.30099,
 12.30033,
 12.29967,
 12.29901,
 12.29837,
 12.29785,
 12.29749,
 12.29718,
 12.29689,
 12.2966,
 12.29632,
 12.29603,
 12.29574,
 12.29545,
 12.29515,
 12.29486,
 12.29457,
 12.29432,
 12.29412,
 12.29393,
 12.29374,
 12.29356,
 12.29338,
 12.29319,
 12.293,
 12.29281,
 12.29262,
 12.29244,
 12.29226,
 12.29214,
 12.29211,
 12.2921,
 12.29211,
 12.29212,
 12.29212,
 12.29213,
 12.29214,
 12.29215,
 12.29216,
 12.29217,
 12.29219,
 12.29225,
 12.29237,
 12.29251,
 12.29266,
 12.29282,
 12.29297,
 12.29313,
 12.29328,
 12.29344,
 12.29359,
 12.29374,
 12.29393,
 12.2944,
 12.29522,
 12.29615,
 12.29709,
 12.29806,
 12.29901,
 12.29994,
 12.30088,
 12.3018,
 12.3027,
 12.30361,
 12.30452,
 12.30559,
 12.30682,
 12.30808,
 12.30933,
 12.31058,
 12.31182,
 12.31304,
 12.31424,
 12.31543,
 12.31661,
 12.31778,
 12.31

In [113]:
results = []
first_level = True
first_flow = True

for f in os.listdir(result_folder):
    if f[-6:]=='.res1d' and not 'ADDOUT' in f and not 'RR' in f and not 'UserSpecified' in f:
        resultdata = Res1D_v5(result_folder + '\\' + f)
        reaches = resultdata.resultData.Reaches
        nodes = resultdata.resultData.Nodes
        
        print ("Importing network " + f + " at " + str(dt.datetime.now()))

#         missing_muids = set()

#         #Check for missing MUIDs in the result file
#         muid_checks = set()

#         rid_checks = set()
#         for i, reach in enumerate(reaches):
#             rid_checks.add(reach.Id[:(-1 * len(str(i)) - 1)])
#         for rid in network_import_specs["MUID_Extended"].values:
#             if not rid in rid_checks:
#                 muid_checks.add(rid)
#         if len(muid_checks)>0:
#             print('In Network_Import_Specs but not in re: ' + str(muid_checks))
#             break

        for i, node in enumerate(nodes):
            
            rid = node.Id
            if rid in level_nodes:
                print ("Importing node " + node.Id + " at " + str(dt.datetime.now()))
                for di in node.DataItems:
                    if di.Quantity.Id.lower() == 'waterlevel':

                        values = list(di.CreateTimeSeriesData(0))
                        level_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        level_df['ResultFile'] = f
                        level_df['MUID'] = rid
                        level_df['Level'] = values                          

                        level_df['DateTimeRef'] = level_df.index

                        if first_level == True:
                            level_df_all = level_df.copy()
                            
                        else:
                            level_df_all = pd.concat([level_df_all,level_df])
                        first_level = False
        
        
        first_round = True
        for i, reach in enumerate(reaches):
            rid = reach.Id[:(-1 * len(str(i)) - 1)]
            gridpoints = reach.GridPoints.Count
            levelpoints = (gridpoints + 1)/2

            if rid in us_level_pipes or rid in ds_level_pipes:
            
                print ("Importing pipe " + reach.Id + " level at " + str(dt.datetime.now()))
                print (gridpoints)
                for di in reach.DataItems:
                    if di.Quantity.Id.lower() == 'waterlevel':

                        if rid in us_level_pipes: 
                            values = list(di.CreateTimeSeriesData(0))
                        else:
                            values = list(di.CreateTimeSeriesData(levelpoints - 1))
                            
                        
                        level_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        level_df['ResultFile'] = f
#                         flow_df['MUID'] = network_import_specs[network_import_specs['MUID_Extended']==rid].iloc[0]['MUID']
                        level_df['MUID'] = rid
                        level_df['Level'] = values                          

                        level_df['DateTimeRef'] = level_df.index

                        if first_level == True:
                            level_df_all = level_df.copy()
                            
                        else:
                            level_df_all = pd.concat([level_df_all,level_df])
                        first_level = False
            
        for i, reach in enumerate(reaches):
            
            rid = reach.Id[:(-1 * len(str(i)) - 1)]
            gridpoints = reach.GridPoints.Count
            if rid in flow_pipes or rid in list(summation_df.MUID):
                print ("Importing pipe " + reach.Id + " discharge at " + str(dt.datetime.now()))
                for di in reach.DataItems:
                    if di.Quantity.Id.lower() == 'discharge':

                        values = list(di.CreateTimeSeriesData(0))
                        flow_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        flow_df['ResultFile'] = f
                        flow_df['MUID'] = rid
                        flow_df['Discharge'] = values                          
                        flow_df['Discharge'] = flow_df['Discharge'] * 1000

                        if rolling_average == True:
                            flow_df['Discharge_Hourly'] = flow_df['Discharge'].rolling('1h').mean()

                        flow_df['DateTimeRef'] = flow_df.index

                        if first_flow == True:
                            flow_df_all = flow_df.copy()

                        else:
                            flow_df_all = pd.concat([flow_df_all,flow_df])
                        first_flow = False

        first_round = False
        
# #Summation
# df_result_sum = pd.merge(flow_df_all,summation_df,how='inner',on=['MUID'])
# df_result_sum = df_result_sum.groupby(['ResultFile','SUMTO','DateTimeRef']).agg({'Discharge':'sum'})
# df_result_sum.reset_index(inplace=True)
# df_result_sum.rename(columns = {'SUMTO':'MUID'},inplace=True)
# flow_df_all = pd.concat([flow_df_all,df_result_sum])


Importing network NSSA_DWF_Cal_2018-07-26_Base.res1d at 2022-10-25 09:09:04.205148
Importing node 10497 at 2022-10-25 09:09:04.205148
Importing node 10498 at 2022-10-25 09:09:04.210152
Importing node 1_119 at 2022-10-25 09:09:04.213181
Importing node 6071 at 2022-10-25 09:09:04.217185
Importing node 6124 at 2022-10-25 09:09:04.220188
Importing node 6819 at 2022-10-25 09:09:04.225212
Importing node 6822 at 2022-10-25 09:09:04.228215
Importing node 7133 at 2022-10-25 09:09:04.231242
Importing node 7169 at 2022-10-25 09:09:04.234245
Importing node 7317 at 2022-10-25 09:09:04.238248
Importing node 9488 at 2022-10-25 09:09:04.241278
Importing node 9525 at 2022-10-25 09:09:04.244281
Importing node 9551 at 2022-10-25 09:09:04.247283
Importing node 9561 at 2022-10-25 09:09:04.250309
Importing node 9600 at 2022-10-25 09:09:04.253311
Importing node 9619 at 2022-10-25 09:09:04.256313
Importing node 9624 at 2022-10-25 09:09:04.259316
Importing node 9629 at 2022-10-25 09:09:04.262343
Importing node

Importing network NSSA_DWF_Cal_2018-08-25_Base.res1d at 2022-10-25 09:09:06.144816
Importing node 10497 at 2022-10-25 09:09:06.144816
Importing node 10498 at 2022-10-25 09:09:06.148819
Importing node 1_119 at 2022-10-25 09:09:06.152823
Importing node 6071 at 2022-10-25 09:09:06.156827
Importing node 6124 at 2022-10-25 09:09:06.160831
Importing node 6819 at 2022-10-25 09:09:06.165835
Importing node 6822 at 2022-10-25 09:09:06.169838
Importing node 7133 at 2022-10-25 09:09:06.173842
Importing node 7169 at 2022-10-25 09:09:06.179847
Importing node 7317 at 2022-10-25 09:09:06.182850
Importing node 9488 at 2022-10-25 09:09:06.186854
Importing node 9525 at 2022-10-25 09:09:06.190858
Importing node 9551 at 2022-10-25 09:09:06.194861
Importing node 9561 at 2022-10-25 09:09:06.198864
Importing node 9600 at 2022-10-25 09:09:06.201867
Importing node 9619 at 2022-10-25 09:09:06.205871
Importing node 9624 at 2022-10-25 09:09:06.209875
Importing node 9629 at 2022-10-25 09:09:06.213878
Importing node

Importing network NSSA_WWF_Cal_2018-11-25_Base.res1d at 2022-10-25 09:09:07.968828
Importing node 10497 at 2022-10-25 09:09:07.968828
Importing node 10498 at 2022-10-25 09:09:07.973857
Importing node 1_119 at 2022-10-25 09:09:07.977861
Importing node 6071 at 2022-10-25 09:09:07.982891
Importing node 6124 at 2022-10-25 09:09:07.986895
Importing node 6819 at 2022-10-25 09:09:07.991925
Importing node 6822 at 2022-10-25 09:09:07.995928
Importing node 7133 at 2022-10-25 09:09:07.999932
Importing node 7169 at 2022-10-25 09:09:08.003960
Importing node 7317 at 2022-10-25 09:09:08.007963
Importing node 9488 at 2022-10-25 09:09:08.011992
Importing node 9525 at 2022-10-25 09:09:08.015995
Importing node 9551 at 2022-10-25 09:09:08.021000
Importing node 9561 at 2022-10-25 09:09:08.025004
Importing node 9600 at 2022-10-25 09:09:08.029007
Importing node 9619 at 2022-10-25 09:09:08.033037
Importing node 9624 at 2022-10-25 09:09:08.037041
Importing node 9629 at 2022-10-25 09:09:08.041070
Importing node

Importing network NSSA_WWF_Cal_2019-01-02_Base.res1d at 2022-10-25 09:09:09.834453
Importing node 10497 at 2022-10-25 09:09:09.834453
Importing node 10498 at 2022-10-25 09:09:09.840459
Importing node 1_119 at 2022-10-25 09:09:09.844462
Importing node 6071 at 2022-10-25 09:09:09.849467
Importing node 6124 at 2022-10-25 09:09:09.854471
Importing node 6819 at 2022-10-25 09:09:09.860477
Importing node 6822 at 2022-10-25 09:09:09.865481
Importing node 7133 at 2022-10-25 09:09:09.869485
Importing node 7169 at 2022-10-25 09:09:09.874490
Importing node 7317 at 2022-10-25 09:09:09.879494
Importing node 9488 at 2022-10-25 09:09:09.883498
Importing node 9525 at 2022-10-25 09:09:09.888502
Importing node 9551 at 2022-10-25 09:09:09.894508
Importing node 9561 at 2022-10-25 09:09:09.898511
Importing node 9600 at 2022-10-25 09:09:09.903516
Importing node 9619 at 2022-10-25 09:09:09.907520
Importing node 9624 at 2022-10-25 09:09:09.912524
Importing node 9629 at 2022-10-25 09:09:09.917529
Importing node

Importing pipe 56393-1597 discharge at 2022-10-25 09:09:11.347561
Importing pipe 56394_3-1600 discharge at 2022-10-25 09:09:11.353567
Importing pipe 56401-1605 discharge at 2022-10-25 09:09:11.359572
Importing pipe 56405-1606 discharge at 2022-10-25 09:09:11.366579
Importing pipe Link_15-1612 discharge at 2022-10-25 09:09:11.372584
Importing network NSSA_WWF_Cal_2020-01-29_Base.res1d at 2022-10-25 09:09:12.066933
Importing node 10497 at 2022-10-25 09:09:12.066933
Importing node 10498 at 2022-10-25 09:09:12.072938
Importing node 1_119 at 2022-10-25 09:09:12.078944
Importing node 6071 at 2022-10-25 09:09:12.086951
Importing node 6124 at 2022-10-25 09:09:12.092956
Importing node 6819 at 2022-10-25 09:09:12.099963
Importing node 6822 at 2022-10-25 09:09:12.105968
Importing node 7133 at 2022-10-25 09:09:12.111974
Importing node 7169 at 2022-10-25 09:09:12.119981
Importing node 7317 at 2022-10-25 09:09:12.126987
Importing node 9488 at 2022-10-25 09:09:12.131992
Importing node 9525 at 2022-10

In [100]:
list(summation_df.MUID)

['54280', '55199', '41294_a', '56405']

,ResultFile,MUID,DateTimeRef,Discharge
0,NSSA_DWF_Cal_2018-07-26_Base.res1d,LY7,2018-07-26 00:00:00,184.971800
1,NSSA_DWF_Cal_2018-07-26_Base.res1d,LY7,2018-07-26 00:05:00,0.000000
2,NSSA_DWF_Cal_2018-07-26_Base.res1d,LY7,2018-07-26 00:10:00,0.000000
3,NSSA_DWF_Cal_2018-07-26_Base.res1d,LY7,2018-07-26 00:15:00,0.000000
4,NSSA_DWF_Cal_2018-07-26_Base.res1d,LY7,2018-07-26 00:20:00,186.896800
...,...,...,...,...
18445,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,2020-02-01 23:40:00,127.503012
18446,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,2020-02-01 23:45:00,127.694494
18447,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,2020-02-01 23:50:00,126.869387
18448,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,2020-02-01 23:55:00,125.155301


In [ ]:
# import plotly
# a = np.random.normal(0,1,100)
# b = np.random.normal(-2,5,100)

# c = np.random.normal(0,1,100)
# d = np.random.normal(-2,5,100)

# fig = plotly.tools.make_subplots(rows=2,cols=1)

# trace_rm1 = go.Histogram(x = a, opacity = 0.75, name = 'malignant')
# trace_rm2 = go.Histogram(x = b, opacity = 0.75, name = 'benign')
# fig.append_trace(go.Histogram(x = a, opacity = 0.75, name = 'benign'),1,1)
# fig.append_trace(go.Histogram(x = b, opacity = 0.75, name = 'malignant'),1,1)
# fig.append_trace(go.Histogram(x = c, opacity = 0.75, name = 'benign'),2,1)
# fig.append_trace(go.Histogram(x = d, opacity = 0.75, name = 'malignant'),2,1)
# fig.layout.update(go.Layout(barmode = 'overlay',))

# fig.show()

In [112]:
flow_df_all

,ResultFile,MUID,Discharge,DateTimeRef
2018-07-26 00:00:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,16.468840,2018-07-26 00:00:00
2018-07-26 00:05:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,16.175270,2018-07-26 00:05:00
2018-07-26 00:10:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,15.881900,2018-07-26 00:10:00
2018-07-26 00:15:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,15.589100,2018-07-26 00:15:00
2018-07-26 00:20:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,15.296860,2018-07-26 00:20:00
...,...,...,...,...
18445,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,127.503012,2020-02-01 23:40:00
18446,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,127.694494,2020-02-01 23:45:00
18447,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,126.869387,2020-02-01 23:50:00
18448,NSSA_WWF_Cal_2020-01-29_Base.res1d,NVLS5,125.155301,2020-02-01 23:55:00


In [148]:
gauges

,Pipe,CSO1,CSO2,Location,Node1 (Or Pipe if pipe level),Node2,CSOPipe1,CSOPipe2,Rain Gauge,Model,Sensor Output,Location Type,Pipe Shape,Pipe Dimension,Node1 AssetName,Flow source,Level Type
Gauge,,,,,,,,,,,,,,,,,
FA2,49582,NaN,NaN,FA2,9600,NaN,NaN,NaN,DN72,NSSA-E,Level and Flow,Gauge,Circular,0.6,MH12A,Measured,Node
GE1,50924,NaN,NaN,GE1,GE1-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,,Measured,Node
GE2,56296,NaN,NaN,GE2,GE2-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,NaN,Measured,Node
GE3,50933,NaN,NaN,GE3,GE3-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
GE4,50940,NaN,NaN,GE5,GE4-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
GE5,50952,NaN,NaN,GE5,GE5-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
GN14,49709,NaN,NaN,GN14,7169,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,Gauge,Circular,0.45,MH14,Measured,Node
HL1,43953,NaN,NaN,HL1,9721,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.9,MH3,Measured,Node
HL14,43945,NaN,NaN,HL14,9712,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.75,MH14,Measured,Node


In [153]:
gauges.columns

Index(['Pipe', 'CSO1', 'CSO2', 'Location', 'Node1 (Or Pipe if pipe level)',
       'Node2', 'CSOPipe1', 'CSOPipe2', 'Rain Gauge', 'Model', 'Sensor Output',
       'Location Type', 'Pipe Shape', 'Pipe Dimension', 'Node1 AssetName',
       'Flow source', 'Level Type'],
      dtype='object')

In [177]:
error_list = []
for gauge in gauge_ids[:1]:
    
#     try:
    if 1 == 1:
        
        zone = str(gauges.loc[gauge,'Location'])
        pipe = str(gauges.loc[gauge,'Pipe'])
        node = str(gauges.loc[gauge,'Node1 (Or Pipe if pipe level)'])
        rain_gauge = str(gauges.loc[gauge,'Rain Gauge'])
        MonitoringType = str(gauges.loc[gauge,"Sensor Output"])
        NodeID = str(gauges.loc[gauge, "Node1 (Or Pipe if pipe level)"])
        MHName = str(gauges.loc[gauge,'Node1 AssetName'])       

        PipeShape = str(gauges.loc[gauge,'Pipe Shape'])
        PipeDimension = str(gauges.loc[gauge, 'Pipe Dimension'])


        with open(output_folder + "\\" + "Calibration_Report_" + gauge + ".html", 'w') as f:
            
            f.write('<style>')
#             f.write('@page {margin-left: 5cm;}')
            f.write('p    {width:1000px;}')
            f.write('p    { font-size: 20px;}')
            f.write('table, th, td {')
            f.write('  border: 1px solid;')
            f.write('  padding: 4px;')
            f.write('  border-collapse: collapse;')
            

            f.write('})')            
            f.write('</style>')
            f.write('<h1 style="text-align:center">Model Calibration Report for gauge ' 
                    + gauge + ' in Zone ' + zone + ', ' + model_area + '</h1>')

            f.write('<h2>1. Description of the Calibration Area</h2>')
            f.write('<h2>2. Description of the Sensor and Network</h2>')            
            f.write('<table>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Monitoring Type' +'</td>\n')
            f.write('<td>'+ MonitoringType +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Node ID' +'</td>\n')
            f.write('<td>'+ node +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe ID' +'</td>\n')
            f.write('<td>'+ pipe +'</td>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe Shape' +'</td>\n')
            f.write('<td>'+ PipeShape +'</td>\n')
            f.write('</tr>\n')
            f.write('</tr>\n')
            f.write('<tr>\n')
            f.write('<td>'+ 'Pipe Dimension (m) or CRS ID ' +'</td>\n')
            f.write('<td>'+ PipeDimension +'</td>\n')
            f.write('</tr>\n')
            f.write('</table>')
    
            
            f.write('<h2>3. Model Calibration Setup: </h2>')
                    
            f.write('<table>')
            f.write('  <tr>')
            f.write('    <th>Simulation</th>')
            f.write('    <th>Start Date</th>')
            f.write('    <th>End Date</th>')
            f.write('    <th>Duration</th>')
            f.write('  </tr>')
                      
            for period_spec in period_specs:
                description = period_spec[:2] + ' Calibration'
                start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
                end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]
                duration = str((end - start).days) + ' days'
                f.write('  <tr>')
                f.write('    <td>' + description + '</th>')
                f.write('    <td>' + start.strftime("%Y-%m-%d %H:%M") + '</th>')
                f.write('    <td>' + end.strftime("%Y-%m-%d %H:%M") + '</th>')
                f.write('    <td>' + duration + '</th>')
                f.write('  </tr>')
                    
            f.write('</table>')
                                      
            f.write('<h2>4. Zone Map</h2>')
            f.write('<img src="' + map_folder + '\\' + zone + '.jpg" alt="' + gauge + '">')

            f.write('<h2>5. DWF Calibration Summary</h2>')  
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['DWF Text'])[0]) + '</p>')

            f.write('<h2>6. WWF Calibration and Validation Summary</h2>')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['WWF Text'])[0]) + '</p>')

            f.write('<h2>7. Calibration Issues</h2>')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Issue Text'])[0]) + '</p>')

            f.write('<h2>8. Recommendations</h2>')
            f.write('<p>' + str(list(report_text[report_text.Zone==zone]['Recommendation Text'])[0]) + '</p>')

            f.write('<h2>9. DWF Calibration Plots</h2>')

            wwf_plot_header_added = False
            for period_spec in period_specs:

                if period_spec[:3] == "WWF" and wwf_plot_header_added == False:
                    f.write('<h2>10. WWF Calibration Plots</h2>')
                    wwf_plot_header_added = True

                start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
                end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]

                chart_header = period_spec[:3] + ', ' + start.strftime("%b %d %Y") + ' - ' + end.strftime("%b %d %Y")

                measured_gauge = measured.loc[(measured.Gauge==gauge) & (measured.index >= start) & (measured.index <= end)]

                measured_rain = rainfall.loc[(rainfall.index >= start) & (rainfall.index <= end)][rain_gauge].to_frame()

                result_flow_gauge = flow_df_all.loc[(flow_df_all.MUID==pipe) & (flow_df_all.index >= start) & (flow_df_all.index <= end)]

                result_level_gauge = level_df_all.loc[(level_df_all.MUID==node) & (level_df_all.index >= start) & (level_df_all.index <= end)]

            #     result_gauge = df_result.loc[(df_result.index >= start) & (df_result.index <= end)]

                fig = plotly.subplots.make_subplots(rows=1,cols=2,subplot_titles=('Discharge',  'Water Level'),specs=[[{"secondary_y": True}, {"secondary_y": True}]])
            #     fig = make_subplots(specs=[[{'secondary_y': True}]])

                fig.add_trace(go.Scatter(x=result_flow_gauge.DateTimeRef, y=result_flow_gauge.Discharge, mode='lines',name='Model Flow'),1,1)   
                fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Flow, mode='lines',name='Measured Flow'),1,1)
                fig.add_trace(go.Scatter(x=measured_rain.index, y=measured_rain[rain_gauge], mode='lines',name='Rainfall'),1,1, secondary_y=True)


                fig.add_trace(go.Scatter(x=result_level_gauge.index, y=result_level_gauge.Level, mode='lines',name='Model Level'),1,2)
                fig.add_trace(go.Scatter(x=measured_gauge.index, y=measured_gauge.Level, mode='lines',name='Measured Level'),1,2)

                fig.update_layout(
                    autosize=False,
                    width = 1600,
                    height=250,
                    margin=dict(
                        l=0,
                        r=0,
                        b=25,
                        t=60,
                        pad=4
                        ),
            #         yaxis_title="Dicharge (L/s)",
                    title ={
                        'text' : chart_header,
                        'x':0.45,
                        'xanchor': 'center'
                    })

                fig['layout']['yaxis']['title']='Dicharge (L/s)'
                fig['layout']['yaxis2']['title']='Rainfall (mm / 5 min)' 
            #     fig['layout']['yaxis2']['autorange']='reversed'
                fig['layout']['yaxis2']['range']=[10,0]
                fig['layout']['yaxis3']['title']='Water Level (m)'



            #     fig.update_yaxes(rangemode="tozero")    

#                 fig.show()
                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))


            #             f.write('<h1>Long Term Simulation ' + w1.value + '</h1>')
            #             f.write('<h2>Model Result</h1>')

        f.close()
        print("Writing html for " + gauge)
#     except Exception as e:
#         error_list.append([gauge_id,e])

    

Writing html for FA2


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime
from plotly.subplots import make_subplots
np.random.seed(123)

fig = make_subplots(specs=[[{'secondary_y': True}]])

n=8

d = {'date': [datetime(2020, 3, k ) for k in range(2, n+2)],
     'A': 21+12*np.random.rand(n),
     'B': 16+12*np.random.rand(n),
     'C': 14+10*np.random.rand(n)}
df = pd.DataFrame(d)

fig.add_trace(go.Scatter(x=df['date'], y= df['A']), secondary_y=False)  
fig.add_trace(go.Scatter(x=df['date'], y= df['B']), secondary_y=False)  

fig.add_trace(go.Scatter(x=df['date'], y= df['C']), secondary_y=True) 


In [51]:
gauges
    

,Gauge,Pipe,CSO1,CSO2,Location,Node1 (Or Pipe if pipe level),Node2,CSOPipe1,CSOPipe2,Rain Gauge,Model,Sensor Output,Location Type,Pipe Shape,Pipe Dimension,Node1 AssetName,Flow source,Level Type
223,FA2,49582,NaN,NaN,FA2,9600,NaN,NaN,NaN,DN72,NSSA-E,Level and Flow,Gauge,Circular,0.6,MH12A,Measured,Node
224,GE1,50924,NaN,NaN,GE1,GE1-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,,Measured,Node
225,GE2,56296,NaN,NaN,GE2,GE2-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,NaN,Measured,Node
226,GE3,50933,NaN,NaN,GE3,GE3-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
227,GE4,50940,NaN,NaN,GE5,GE4-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
228,GE5,50952,NaN,NaN,GE5,GE5-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
229,GN14,49709,NaN,NaN,GN14,7169,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,Gauge,Circular,0.45,MH14,Measured,Node
230,HL1,43953,NaN,NaN,HL1,9721,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.9,MH3,Measured,Node
231,HL14,43945,NaN,NaN,HL14,9712,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.75,MH14,Measured,Node
232,HL18,49559,NaN,NaN,HL14,9708,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.55,MH18,Measured,Node
